In [ ]:
#culmination of our work
from DendroHeat import DendroHeat
dh = DendroHeat()
dh.addData(file='HexaPropsAssay.xlsx', type='raw')
dh.addData(file='HexaLow_matrix.csv', type='cluster')
dh.show()

In [20]:
#get packages we will use
from math import pi
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool#, LogColorMapper
# from bokeh.palettes import Viridis6 as palette
from bokeh.layouts import layout
output_notebook()

from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from scipy.spatial.distance import squareform

Loading BokehJS ...

In [ ]:
#import data and make a normalized copy of it
df = pd.read_excel('HexaPropsAssay.xlsx', index_col=0)
cdtree = pd.read_csv('cdtree.csv')
ndf = (df - df.min(axis=0)) / (df.max(axis=0) - df.min(axis=0))

In [ ]:
# print(sum(cdtree['child_size'].loc[838]))
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, gen_min_span_tree=True)
clusterer.fit(df.fillna(0))
clusterer.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

In [ ]:
#create dendrogram tree via dicitonary
import collections
parents = cdtree['parent'].tolist()
children = cdtree['child'].tolist()
forwardtree = collections.defaultdict(set)
reversetree = {}
for x,y in zip(parents,children):
    reversetree[y] = x
    forwardtree[x].add(y)
for i in range(0, 838):
    forwardtree[i].add('')

In [ ]:
parentdepth = collections.defaultdict(set) # depth : set of all parent nodes at depth
leafdepth = collections.defaultdict(set) # depth: set of all leaves at depth
rleafdepth = {} # leaf:depth

#use DFS to find depth of each node
def dfs(graph, start, depth, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    for next in graph[start] - visited:
#         print(next)
        if forwardtree[next]:
            parentdepth[depth].add(next)
        else:
            leafdepth[depth].add(next)
            rleafdepth[next] = depth
        dfs(graph, next, depth+1, visited)
#     return visited

dfs(forwardtree, 838, 1)
# sorted based on depth
leaves = sorted(range(0, 838), key = lambda x:rleafdepth[x])

# while nodes_to_visit:
#     depth = depth + 1
#     currentnode = nodes_to_visit.pop()
#     nodes_to_visit = forwardtree[currentnode] + nodes_to_visit
#     if not forwardtree[currentnode]:
#         depthtree[depth].append(currentnode)
# #         breaktracker[depth].add(reversetree[currentnode])
#         depthtracker[currentnode] = depth
#         depth = depth - 1
#     else:
#         breaktracker[depth-1].add(currentnode)

# print(depthtree)
# print(breaktracker)
# ordered = sorted(range(0,838), key = lambda x:depthtracker[x], reverse = True)

In [ ]:

xs=[]
ys=[]
name=[]


# yrange = {}
# depth = max(parentdepth)

# at each depth, figure out the y range that each parent overlooks. then check if it intersects with any other
# parent at it's depth. If so, move to the closest end and space up/down as needed.
# while depth > 0:
#     for p in parentdepth[depth]:
#         if for x in forwardtree[p] < 838:
#             yrange[p] = [leaves.index(min(forwardtree[p])), leaves.index(max(forwardtree[p]))]
#     for p in parentdepth[depth]:
#         for x in parentdepth[depth]:
#             bot, top = False
#             if x is not p:
#                 if yrange[p][0] in range(yrange[x][0], yrange[x][1]+1):
#                     bot = True
#                 if yrange[p][0] in range(yrange[x][0], yrange[x][1]+1):
#                     top = True
#                 if top and bottom:
#                     # if parent p is fully inside parent x
#                     choices = [yrange[p][1] - yrange[x][0] + 1, yrange[x][1] - yrange[p][0] + 1]
#                     closest = min(choices)
                    
#                 elif top:
#                     # if only the top is inside x
#                     movedown = yrange[p][1] - yrange[x][0] + 1
#                     for x in yrange[p]:
#                         yrange[p][x] = x - movedown
#                 elif bot:
#                     # if only the bottom is inside x
#                     moveup = yrange[x][1] - yrange[p][0] + 1
#                     for x in yrange[p]:
#                         yrange[p][x] = x + moveup
#     depth = depth-1
    
yi=.5
for x in leaves:
    low = yi
    xs.append([rleafdepth[x], 200])
    ys.append([yi, yi])
    name.append([df.index[y]])
    yi = yi+1
#     xs.append([x, x])
#     ys.append([low, yi])
#     name.append('lsl')


dendro = figure(x_range = [0,40], y_range = [0,df.shape[0]+.5])
dendro.multi_line(xs='xs', ys='ys', source=ColumnDataSource(dict(xs=xs, ys=ys, name=name)))
h = HoverTool()
h.tooltips = [("Molecule", "@name")]
dendro.add_tools(h)


dendro.grid.grid_line_color = None
dendro.axis.axis_line_color = None
show(dendro)

In [ ]:
#reshape the data to be easily understood by bokeh
molecule = []
attribute = []
xs = []
ys = []
crange = ['#66ff66', '#00ffff', '#000099', '#6600ff', '#ff00ff', '#990099', '#ff0066', '#993333', '#ff9933', '#ffff00']
colors = []
alpha = []
value = []

# df = df.iloc[ordered]

for i, m in enumerate(df.index):
    molecule = molecule + [m]*len(df.columns)
    attribute = attribute + df.columns.tolist()
    colors = colors + crange*2
    xs = xs + list(np.arange(0.5, len(df.columns)+0.5))
    ys = ys + [i+0.5]*len(df.columns)
    value = value + df.loc[m].tolist()
    alpha = alpha + ndf.loc[m].tolist()

data = pd.DataFrame(dict(
        molecule = molecule,
        attribute = attribute, 
        colors = colors,
        xs = xs,
        ys = ys,
        value = value,
        alpha = alpha
              ))
# color_mapper = LogColorMapper(palette=palette)
source = ColumnDataSource(data)

In [ ]:
#create tooltip and heatmap
hover = HoverTool()

hover.tooltips = [("Molecule", "@molecule"), ("Attribute", "@attribute"), ("Value", "@value")]

hm = figure(x_range=df.columns.tolist(), y_range=[0, df.shape[0]])
hm.rect(x='xs', y='ys', height=1, width=1, source=source, fill_alpha='alpha', line_alpha=0.1, fill_color='colors')
# for i, d in zip(icoord, dcoord):
#     d = list(map(lambda x: -x, d))
#     hm.line(x=d, y=i, line_color = 'black')

hm.xaxis.major_label_orientation = pi/2
hm.xaxis.major_label_text_font_size = '7pt'
hm.axis.major_tick_line_color = None
hm.axis.minor_tick_line_color = None
hm.yaxis.major_label_text_color = None
hm.grid.grid_line_color = None
hm.axis.axis_line_color = None

hm.add_tools(hover)
show(hm)

In [25]:
a = pd.read_csv("HexaLow_matrix.csv", index_col=0)
b = linkage(a, method='ward')
pd.DataFrame(b).to_excel("HexaLinkage_matrix_ward.xlsx")